<div style="text-align: center;">
    <h1>Codes Nomenclature des Activités Françaises (NAF) </h1>
</div>


Le but de ce notebook est de télécharger une liste de code NAF.

La NAF, nomenclature d'activités française, est une nomenclature des activités économiques productives, principalement élaborée pour faciliter l'organisation de l'information économique et sociale.

Le code NAF (ou APE) est construit de la manière suivante :

<p align="center">
  <img src="../img/codeape.png" alt="imagemanquante" width="30%" />
</p>



Plus d'informations sur le site de l'[INSEE](https://www.insee.fr/fr/information/2406147).

Il s'agit d'extraire de ces données les colonnes suivantes :

- codeNAF : on prendra la version rev2 de la nomenclature
- Intitule : intitulé du code NAF



In [6]:
! pip install -q xlrd


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import os 
import pandas as pd 
import re
import numpy as np

# 1. Téléchargement des données


Les codes Naf fournis par l'INSEE peuvent étre téléchargés sur [ce lien](https://www.insee.fr/fr/statistiques/fichier/2120875/int_courts_naf_rev_2.xls). 


In [8]:
os.system("wget -q -P '../Données nationales/' https://www.insee.fr/fr/statistiques/fichier/2120875/int_courts_naf_rev_2.xls")

0

In [9]:
file_path = "../Données nationales/int_courts_naf_rev_2.xls"

df = pd.read_excel(file_path)

df

,ligne,Code,"Intitulés de la NAF rév. 2, version finale","Intitulés NAF rév. 2, \nen 65 caractères","Intitulés NAF rév. 2, \nen 40 caractères"
0,1,NaN,NaN,NaN,NaN
1,2,SECTION A,"AGRICULTURE, SYLVICULTURE ET PÊCHE","AGRICULTURE, SYLVICULTURE ET PÊCHE","AGRICULTURE, SYLVICULTURE ET PÊCHE"
2,3,NaN,NaN,NaN,NaN
3,4,01,"Culture et production animale, chasse et servi...","Culture et production animale, chasse et servi...","Cult. & prod. animale, chasse & sce ann."
4,5,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2104,2105,99,Activités des organisations et organismes extr...,Activités des organisations et organismes extr...,Act. organisations extraterritoriales
2105,2106,NaN,NaN,NaN,NaN
2106,2107,99.0,Activités des organisations et organismes extr...,Activités des organisations et organismes extr...,Act. organisations extraterritoriales
2107,2108,99.00,Activités des organisations et organismes extr...,Activités des organisations et organismes extr...,Act. organisations extraterritoriales


# 2. Nettoyage des données

Dans la colonne 'Code', toutes les mailles sont présentes (Division / Classe et sous Classe).

On ne garde que la maille la plus fine de ce découpage, c'est-à-dire les codes de la forme "99.00Z".

In [10]:
# function to match the pattern
def match(string):
    return bool(re.match(r"\d{2}\.\d{2}[A-Z]", string))

#converts Code to string
df["Code"] = df["Code"].astype(str)

#apply the function to the column
df["match"] = df["Code"].apply(match)

#filter the dataframe
df = df[df["match"] == True]


In [11]:
df.columns

Index(['ligne', 'Code', ' Intitulés de la  NAF rév. 2, version finale ',
       'Intitulés NAF rév. 2, \nen 65 caractères',
       'Intitulés NAF rév. 2, \nen 40 caractères', 'match'],
      dtype='object')

## Sélection des variables d'intérêt

On veut garder :
- codeNaf 
- libelle

In [12]:
df = df[df.columns[1:3]]

## Sauvegarde du fichier

In [13]:
df.columns = ['codeNaf','libelleNaf']

df.to_parquet("../Données nationales/NAF.parquet", index=False)

# 3. Constitution d'un sous-groupe de code NAF utile pour FoodBiome

La nomenclature NAF recouvre un grand nombre de secteurs, pour notre application seulement une partie sera utile.

On sélectionne à la main ceux dont on pense qu'ils peuvent être utiles.

- Le [premier document](../Données%20nationales/NAF_FoodBiome.xlsx) correspond à un premier filtre fait par votre serviteur.
- Le [second document](../Données%20nationales/NAF_FoodBiome_a_modifier.xlsx) est une liste de tous les codes NAF et peut-être modifié par n'importe qui.

*Pour le modifier je suggère de :
- dupliquer le fichier "NAF_FoodBiome_a_modifier.xslx"
- ouvrir le fichier dupliqué
- enlever les lignes inutiles
- renommer le fichier "NAF_FoodBiome.xlsx"

In [15]:
df.to_excel("../Données nationales/NAF_FoodBiome.xlsx", index=False)
df.to_excel("../Données nationales/NAF_FoodBiome_a_modifier.xlsx", index=False)